# header file included

In [1]:
import pandas as pd
import numpy as np
import seaborn as sb
import matplotlib.pyplot as plt

print('done')

done


# data loading

In [2]:
data = pd.read_csv('preprocessed_data.csv')
data.head()

,Video id,Title,Description,Category
0,DU1BEo2GlnM,day gulftain missil travel iran ferri,https www timesofisrael com former mk say iran...,travel
1,XatWIV-bWnA,travel hack collab heyitsjemma,hey hope guy enjoy know video bit differ fit h...,travel
2,p0-Sa6hK9ZU,kh nh linh travel h ng th ng gia bamboo airway...,kh nh linh travel vlog h ng th ng gia bamboo a...,travel
3,egjqs_6zvno,best video lefkada holiday greec travel drone ...,subskrybuj j kana instagram https www instagra...,travel
4,abH6pFIp--Y,vlog travel pack storm new addict matur beauti...,hectic week pack teach get pedicur get easter ...,travel


In [3]:
data.dropna(inplace=True)

In [4]:
data.shape

(1868, 4)

# dropping 'Video id' column

In [5]:
# dropping 'Video id' columns as it is not needed
data = data.drop('Video id', axis=1)
data.head(5)

,Title,Description,Category
0,day gulftain missil travel iran ferri,https www timesofisrael com former mk say iran...,travel
1,travel hack collab heyitsjemma,hey hope guy enjoy know video bit differ fit h...,travel
2,kh nh linh travel h ng th ng gia bamboo airway...,kh nh linh travel vlog h ng th ng gia bamboo a...,travel
3,best video lefkada holiday greec travel drone ...,subskrybuj j kana instagram https www instagra...,travel
4,vlog travel pack storm new addict matur beauti...,hectic week pack teach get pedicur get easter ...,travel


# label conversion

In [6]:
data['Category'] = data.Category.map({'travel':0, 'science':1, 'food':2, 'manufacturing':3, 'history':4, 'art':5})
data.tail(5)

,Title,Description,Category
1865,element lindsey stirl dubstep violin origin song,pick copi delux edit lindsey christma album wa...,5
1866,bebe rexha mess offici music video,new singl debut album expect https beberexha i...,5
1867,medicin queen naija offici video,self titl ep featur karma medicin listen http ...,5
1868,post malon go flex,go flex offici video download today http repub...,5
1869,alan walker differ world feat sofia carson k c...,walker excit final releas new singl differ wor...,5


# combining 'Title' and 'Description' into one column 'TitleDescription'

In [7]:
data['TitleDesciption'] = data['Title']+ ' ' + data['Description']
data.head()

,Title,Description,Category,TitleDesciption
0,day gulftain missil travel iran ferri,https www timesofisrael com former mk say iran...,0,day gulftain missil travel iran ferri https ww...
1,travel hack collab heyitsjemma,hey hope guy enjoy know video bit differ fit h...,0,travel hack collab heyitsjemma hey hope guy en...
2,kh nh linh travel h ng th ng gia bamboo airway...,kh nh linh travel vlog h ng th ng gia bamboo a...,0,kh nh linh travel h ng th ng gia bamboo airway...
3,best video lefkada holiday greec travel drone ...,subskrybuj j kana instagram https www instagra...,0,best video lefkada holiday greec travel drone ...
4,vlog travel pack storm new addict matur beauti...,hectic week pack teach get pedicur get easter ...,0,vlog travel pack storm new addict matur beauti...


# CNN

In [15]:
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam

from keras.layers.embeddings import Embedding
from keras.layers import Flatten, Conv1D, MaxPooling1D, Dropout, Activation, GlobalMaxPooling1D

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

import keras_metrics
from sklearn.metrics import precision_score, recall_score
print('done')

done


In [10]:
vocab_size = 40000
tokenizer = Tokenizer(num_words= vocab_size)
tokenizer.fit_on_texts(data['TitleDesciption'])

maxLen = 250
sequences = tokenizer.texts_to_sequences(data['TitleDesciption'])
data_cnn = pad_sequences(sequences, maxlen=maxLen)

In [11]:
print(data_cnn.shape)
print(data_cnn)

(1868, 250)
[[   0    0    0 ...,    9 3568    7]
 [   0    0    0 ..., 2194   21   31]
 [   0    0    0 ...,   64  140 1705]
 ..., 
 [   0    0    0 ...,  334  125   98]
 [   0    0    0 ...,  152  354 1110]
 [   0    0    0 ..., 1657  192 1658]]


# OneHotEncoder

In [25]:
label = data['Category'].values
print(label.shape)
print(label)

(1868,)
[0 0 0 ..., 5 5 5]


In [29]:
label_onehot = pd.get_dummies(label)
print(label_onehot.shape)
# print(label_onehot)

(1868, 6)


# sequential model

In [38]:
model_conv = Sequential()
model_conv.add(Embedding(vocab_size, 100, input_length=maxLen))
model_conv.add(Dropout(0.2))

model_conv.add(Conv1D(100, 6, activation='relu'))
model_conv.add(Dropout(0.2))

model_conv.add(Conv1D(64, 6, activation='relu'))
model_conv.add(Dropout(0.2))

model_conv.add(Flatten())
model_conv.add(Dense(64, activation='relu'))

model_conv.add(Dropout(0.2))
model_conv.add(Dense(64, activation='relu'))

model_conv.add(Dropout(0.2))
model_conv.add(Dense(6, activation='sigmoid'))

model_conv.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.1), metrics=['accuracy', keras_metrics.precision(), keras_metrics.recall()])#metrics=['accuracy'])

In [33]:
model_conv.fit(data_cnn, label_onehot, batch_size=512, epochs=42)#, validation_split=.2)#, sample_weight=sample_weights)

Epoch 1/42
1868/1868 [==============================] - 11s 6ms/step - loss: 6.0470 - acc: 0.1627 - precision: 0.1627 - recall: 1.0000
Epoch 2/42
1868/1868 [==============================] - 11s 6ms/step - loss: 6.0547 - acc: 0.1627 - precision: 0.1627 - recall: 1.0000
Epoch 3/42
1868/1868 [==========